<a href="https://colab.research.google.com/github/lavlugit/lavlugit/blob/main/Deepseek_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fpdf pandas matplotlib gradio requests

import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
import tempfile
import os
import requests
import json
import time

# ==========================
# DeepSeek API Configuration (FREE)
# ==========================
DEEPSEEK_API_KEY = "sk-1ac538fa013446fd83b6124d806219a9"  # Get free key from: https://platform.deepseek.com/
DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

class DeepSeekAPI:
    def __init__(self):
        self.api_key = DEEPSEEK_API_KEY
        self.configured = self.api_key and self.api_key != "your_deepseek_api_key_here"

    def generate_text(self, prompt, max_tokens=500):
        """Generate text using DeepSeek FREE API"""
        if not self.configured:
            return self._demo_response(prompt)

        try:
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self.api_key}"
            }

            payload = {
                "model": "deepseek-chat",  # Free model
                "messages": [
                    {"role": "system", "content": "You are a helpful academic research assistant. Provide detailed, scholarly content suitable for research papers."},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": max_tokens,
                "temperature": 0.7,
                "stream": False
            }

            response = requests.post(DEEPSEEK_API_URL, headers=headers, json=payload, timeout=60)

            if response.status_code == 200:
                result = response.json()
                return result["choices"][0]["message"]["content"]
            else:
                return f"API Error {response.status_code}. Using demo content."

        except Exception as e:
            return f"DeepSeek API Error: {str(e)}. Using demo content."

    def _demo_response(self, prompt):
        """Quality demo responses"""
        demo_responses = {
            "abstract": "**Abstract**: This research study employs rigorous methodology to investigate the research topic, yielding significant findings that contribute to academic knowledge and practical applications in the field.",
            "introduction": "**Introduction**: This paper establishes a comprehensive theoretical framework and addresses important research gaps through systematic investigation of the topic.",
            "methodology": "**Methodology**: The research design incorporates appropriate methods for data collection and analysis, ensuring validity and reliability of findings.",
            "results": "**Results**: Statistical analysis reveals significant patterns and relationships, providing empirical support for the research hypotheses.",
            "discussion": "**Discussion**: The findings offer valuable insights with important implications for both theoretical understanding and practical applications.",
            "conclusion": "**Conclusion**: This study makes substantive contributions to the field and suggests promising directions for future research."
        }

        prompt_lower = prompt.lower()
        for key, value in demo_responses.items():
            if key in prompt_lower:
                return value + "\n\n*[Add your free DeepSeek API key for AI-generated content]*"

        return f"**Academic Content**: Comprehensive analysis reveals important insights about the research topic.\n\n*[Get free API key from https://platform.deepseek.com/]*"

# Initialize DeepSeek API
deepseek = DeepSeekAPI()

# ==========================
# CrossRef API for References (Free)
# ==========================
def get_references(query, num=5):
    """Get academic references from CrossRef (Free)"""
    try:
        url = f"https://api.crossref.org/works?query={requests.utils.quote(query)}&rows={num}"
        response = requests.get(url, timeout=15)

        if response.status_code != 200:
            return _fallback_references(query)

        items = response.json().get("message", {}).get("items", [])
        if not items:
            return _fallback_references(query)

        refs = ["**Academic References:**"]
        for i, item in enumerate(items, 1):
            title = item.get("title", ["No title"])[0] if item.get("title") else "Untitled"
            authors = item.get("author", [])
            author_names = []

            for author in authors[:2]:
                given = author.get("given", "")
                family = author.get("family", "")
                if given or family:
                    author_names.append(f"{given} {family}".strip())

            author_str = ", ".join(author_names) if author_names else "Various Authors"
            year = item.get("published-print", {}).get("date-parts", [[None]])[0][0]
            year_str = f" ({year})" if year else ""
            journal = item.get("container-title", [""])[0] if item.get("container-title") else ""
            journal_str = f" *{journal}*" if journal else ""

            refs.append(f"{i}. {author_str}{year_str}. {title}.{journal_str}")

        return "\n\n".join(refs)

    except Exception as e:
        return _fallback_references(query)

def _fallback_references(query):
    """Fallback reference template"""
    return f"""**References for: {query}**

1. Comprehensive literature review using Google Scholar
2. Academic database search (IEEE, Springer, Elsevier)
3. Recent journal articles and conference proceedings
4. Relevant theoretical frameworks
5. Empirical studies in the field"""

# ==========================
# Dataset Analysis Functions
# ==========================
def summarize_dataset(df):
    """Generate comprehensive dataset summary"""
    try:
        summary = f"**Dataset Overview:** {df.shape[0]:,} rows × {df.shape[1]} columns\n\n"
        summary += "**Column Analysis:**\n"

        for col in df.columns:
            dtype = str(df[col].dtype)
            missing = df[col].isnull().sum()
            missing_pct = (missing / len(df)) * 100
            unique = df[col].nunique()

            summary += f"• **{col}**: {dtype} | Missing: {missing} ({missing_pct:.1f}%) | Unique: {unique}\n"

        # Numeric columns summary
        numeric_cols = df.select_dtypes(include=["number"]).columns
        if len(numeric_cols) > 0:
            summary += f"\n**Numeric Analysis** ({len(numeric_cols)} columns):\n"
            for col in numeric_cols[:3]:
                summary += f"• {col}: Mean={df[col].mean():.2f}, Std={df[col].std():.2f}\n"

        return summary

    except Exception as e:
        return f"**Dataset Analysis:** Basic structure analyzed."

def generate_plots_and_stats(df):
    """Generate professional visualizations"""
    temp_dir = tempfile.mkdtemp()
    stats_file = None
    plot_files = []

    try:
        # Statistics table
        numeric_df = df.select_dtypes(include=["number"])
        if not numeric_df.empty:
            plt.figure(figsize=(12, 6))
            plt.axis('off')

            stats = numeric_df.describe().round(3)
            table = plt.table(cellText=stats.values,
                             rowLabels=stats.index,
                             colLabels=stats.columns,
                             cellLoc='center',
                             loc='center')
            table.auto_set_font_size(False)
            table.set_fontsize(8)
            table.scale(1, 1.5)

            stats_file = os.path.join(temp_dir, "statistics.png")
            plt.tight_layout()
            plt.savefig(stats_file, bbox_inches='tight', dpi=150)
            plt.close()

        # Visualizations
        numeric_cols = df.select_dtypes(include=["number"]).columns
        for col in numeric_cols[:2]:
            try:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

                # Histogram
                ax1.hist(df[col].dropna(), bins=15, color='lightblue', alpha=0.7, edgecolor='black')
                ax1.set_title(f'Distribution: {col}')
                ax1.set_xlabel(col)
                ax1.grid(alpha=0.3)

                # Box plot
                ax2.boxplot(df[col].dropna())
                ax2.set_title(f'Spread: {col}')
                ax2.set_ylabel(col)
                ax2.grid(alpha=0.3)

                plt.tight_layout()
                plot_file = os.path.join(temp_dir, f"{col}_analysis.png")
                plt.savefig(plot_file, bbox_inches='tight')
                plt.close()
                plot_files.append(plot_file)

            except Exception as e:
                continue

    except Exception as e:
        print(f"Visualization generation: {e}")

    return stats_file, plot_files

# ==========================
# PDF Generation
# ==========================
class ResearchPDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Research Paper (DeepSeek AI)', 0, 0, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

def build_pdf(topic, sections, paper, pdf_file_path, stats_image=None, plot_files=None):
    """Build research PDF"""
    try:
        pdf = ResearchPDF()

        # Title page
        pdf.add_page()
        pdf.set_font('Arial', 'B', 18)
        pdf.cell(0, 20, 'ACADEMIC RESEARCH PAPER', 0, 1, 'C')
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 15, topic, 0, 1, 'C')
        pdf.ln(10)
        pdf.set_font('Arial', 'I', 10)
        pdf.cell(0, 10, 'Generated with DeepSeek AI Assistant', 0, 1, 'C')

        # Content
        for section in sections:
            if section == "Title":
                continue

            pdf.add_page()
            pdf.set_font('Arial', 'B', 14)
            pdf.cell(0, 10, section.upper(), 0, 1)
            pdf.ln(5)

            pdf.set_font('Arial', '', 12)
            content = paper.get(section, '')
            pdf.multi_cell(0, 8, content)

            # Visualizations for Results section
            if section == "Results":
                if stats_image and os.path.exists(stats_image):
                    pdf.ln(10)
                    pdf.image(stats_image, x=10, w=190)

                if plot_files:
                    for plot_file in plot_files or []:
                        if plot_file and os.path.exists(plot_file):
                            try:
                                pdf.image(plot_file, x=10, w=190)
                                pdf.ln(5)
                            except:
                                pass

        pdf.output(pdf_file_path)
        return True

    except Exception as e:
        print(f"PDF generation error: {e}")
        return False

# ==========================
# Main Research Paper Generator with DeepSeek
# ==========================
def generate_paper(topic, keywords, dataset_file=None):
    """Generate research paper using DeepSeek API"""
    start_time = time.time()
    print("🚀 DeepSeek Research Paper Generator Started...")

    try:
        # Research paper structure
        sections = ["Abstract", "Introduction", "Literature Review",
                   "Methodology", "Results", "Discussion", "Conclusion"]
        paper = {"Title": f"Research: {topic}"}

        # Dataset processing
        dataset_summary = "No dataset provided."
        stats_image, plot_files = None, []

        if dataset_file is not None:
            try:
                print("📊 Analyzing dataset...")
                df = pd.read_csv(dataset_file.name)
                df = df.iloc[:, :10]  # Limit columns for performance
                dataset_summary = summarize_dataset(df)
                stats_image, plot_files = generate_plots_and_stats(df)
                print("✅ Dataset analysis completed")
            except Exception as e:
                print("⚠️ Dataset processing simplified")

        # Generate content with DeepSeek
        print("🤖 Generating academic content with DeepSeek...")

        prompts = {
            "Abstract": f"Write a concise academic abstract about '{topic}'. Keywords: {keywords}. Include objectives, methods, findings, implications.",
            "Introduction": f"Write an introduction for a research paper about '{topic}'. Provide background, problem statement, research questions.",
            "Literature Review": f"Write a literature review section about '{topic}'. Summarize relevant research and identify gaps.",
            "Methodology": f"Describe research methodology for studying '{topic}'. Data context: {dataset_summary}",
            "Results": f"Present research results for '{topic}'. Focus on key findings and data patterns.",
            "Discussion": f"Discuss the implications of research findings about '{topic}'.",
            "Conclusion": f"Write a conclusion summarizing the research on '{topic}' and future directions."
        }

        for section, prompt in prompts.items():
            section_start = time.time()
            paper[section] = deepseek.generate_text(prompt)
            section_time = time.time() - section_start
            print(f"   ✅ {section}: {section_time:.1f}s")

        # References
        paper["References"] = get_references(topic)

        # Generate PDF
        safe_topic = "".join(c for c in topic if c.isalnum() or c in (' ', '-', '_')).rstrip()
        pdf_filename = f"/tmp/DeepSeek_{safe_topic[:20]}_research.pdf"

        all_sections = ["Title"] + sections + ["References"]
        success = build_pdf(topic, all_sections, paper, pdf_filename, stats_image, plot_files)

        total_time = time.time() - start_time
        print(f"✅ GENERATION COMPLETE: {total_time:.1f} seconds")
        print(f"🔑 API Status: {'DeepSeek Connected' if deepseek.configured else 'Demo Mode'}")

        if success:
            return pdf_filename
        else:
            raise Exception("PDF generation failed")

    except Exception as e:
        print(f"❌ Error: {e}")
        # Fallback PDF
        pdf_filename = "/tmp/deepseek_paper.pdf"
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=14)
        pdf.cell(0, 10, f"Research: {topic}", 0, 1)
        pdf.multi_cell(0, 10, "Research paper generated with DeepSeek AI.")
        pdf.output(pdf_filename)
        return pdf_filename

# ==========================
# Gradio Interface
# ==========================
def create_interface():
    with gr.Blocks(title="DeepSeek Research Paper Generator", theme=gr.themes.Soft()) as demo:

        gr.Markdown("""
        # 🚀 DeepSeek Research Paper Generator
        *Free AI-powered academic papers using DeepSeek API*
        """)

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📝 Research Parameters")

                topic = gr.Textbox(
                    label="Research Topic",
                    value="Artificial Intelligence in Education",
                    lines=2
                )

                keywords = gr.Textbox(
                    label="Keywords",
                    value="AI, education, machine learning, technology"
                )

                dataset = gr.File(
                    label="Upload Dataset (Optional)",
                    file_types=[".csv"]
                )

                gr.Markdown("""
                **How to get FREE API key:**
                1. Visit https://platform.deepseek.com/
                2. Sign up for free account
                3. Get your API key from dashboard
                4. Replace the API key in the code
                """)

                generate_btn = gr.Button(
                    "🚀 Generate with DeepSeek",
                    variant="primary"
                )

            with gr.Column():
                output_file = gr.File(label="Download Research Paper")

                with gr.Accordion("ℹ️ DeepSeek API Info", open=False):
                    gr.Markdown("""
                    **DeepSeek Free Tier:**
                    - ✅ Free API access
                    - ✅ Good rate limits
                    - ✅ Quality AI models
                    - ✅ Suitable for research

                    **Current Status:** {'✅ Connected' if deepseek.configured else '⚠️ Demo Mode - Add API Key'}
                    """)

        gr.Examples(
            examples=[
                ["Climate Change Impact on Agriculture", "climate, agriculture, sustainability"],
                ["Digital Health Technologies", "healthcare, technology, telemedicine"]
            ],
            inputs=[topic, keywords]
        )

        generate_btn.click(
            fn=generate_paper,
            inputs=[topic, keywords, dataset],
            outputs=output_file
        )

    return demo

# ==========================
# Launch Application
# ==========================
if __name__ == "__main__":
    print("🚀 Starting DeepSeek Research Paper Generator...")
    print("💡 Get FREE API key from: https://platform.deepseek.com/")
    print(f"🔑 Current API Status: {'Connected' if deepseek.configured else 'Demo Mode - Add Your Free Key'}")

    demo = create_interface()
    demo.launch(share=True)

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=95060f8f73caa2ef4085d6683f8757b36be3f1713dd7e8eb5c71be9e4ab20f84
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf
🚀 Starting DeepSeek Research Paper Generator...
💡 Get FREE API key from: https://platform.deepseek.com/
🔑 Current API Status: Connected
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://72a6af91f7988914af.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
